### Search enumerated structures within the Drugbank database

In [1]:
import pandas as pd
from rdkit import Chem
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

from tqdm.notebook import tqdm
import processing_utils

### Load structures from Drugbank - can be replaced by any other database.
downloaded from https://go.drugbank.com/releases/latest#structures

In [2]:
drugs = Chem.SDMolSupplier("../drugbank_structures.sdf")
drugs = [i for i in drugs if i]
for d in drugs:
    # cleanup
    Chem.SanitizeMol(d)
    
    # Kekulization removed.
#     Chem.Kekulize(d,clearAromaticFlags=True)

In [3]:
data = pd.read_csv("./data_files/smiles_min_dist_natoms.csv",index_col=0)

### run search

In [ ]:

map_input = [[s,drugs,False] for s in data.smiles]

if __name__ == '__main__':
    with mp.Pool(32) as p:
        result = p.starmap(processing_utils.search_in_database, map_input,chunksize=1000)

In [7]:
data["drugbank_matches"] = result

In [8]:
# take a look
data.head()

,smiles,min_dist_all,natoms,drugbank_matches
0,COON,6.0,4,0
1,CONO,6.0,4,18
2,CON=O,7.0,4,19
3,NOCO,4.0,4,4
4,C1ONO1,5.0,4,0


In [9]:
# save to csv as a backup 
data.to_csv("./data_files/smiles_min_dist_dbank.csv")